In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np

import sys
from pathlib import Path

# Add project paths
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))

# Import experiment management from HDMF src folder
from simulation_runner import HDMFSimulationRunner
from experiment_manager import ExperimentManager


def compute_fcd(data, wsize, overlap, isubdiag):
    T, N = data.shape
    win_start = np.arange(0, T - wsize - 1, wsize - overlap)
    nwins = len(win_start)
    fcd = np.zeros((len(isubdiag[0]), nwins))
    #print(fcd.shape)
    #print(data.shape)
    #print((data[win_start[2]:win_start[2] + wsize + 1, :]).shape)
    for i in range(nwins):
        tmp = data[win_start[i]:win_start[i] + wsize + 1, :]
        cormat = np.corrcoef(tmp.T)
        fcd[:, i] = cormat[isubdiag[0],isubdiag[1]]
    return fcd

from utils.data_loading import load_all_sc_matrices


In [4]:
exp = ExperimentManager(project_root=project_root, config_path="Fitting_Grid", results_dir="/network/iss/home/ivan.mindlin/Repos/fastHDMF/results")
#runner = HDMFSimulationRunner(project_root=project_root, experiment_manager=exp)
#res = runner.run_experiment()
#res = exp.load_experiment_results()
runner = HDMFSimulationRunner(project_root=project_root, experiment_manager=exp)


2025-09-22 14:15:46,787 - hdmf_experiment_Fitting_Grid - INFO - Experiment 'Fitting_Grid' initialized
2025-09-22 14:15:46,789 - hdmf_experiment_Fitting_Grid - INFO - Config: FC and FCD grid 
2025-09-22 14:15:46,790 - hdmf_experiment_Fitting_Grid - INFO - Results will be saved to: /network/iss/home/ivan.mindlin/Repos/fastHDMF/results/Fitting_Grid


INFO: Loaded config from: /network/iss/home/ivan.mindlin/Repos/fastHDMF/configs/experiments/Fitting_Grid.yaml
Creating IPP list from SC matrices in: AAL
Found 1 IPPs from SC matrices


In [5]:
item = runner.define_items_over(exp.current_config)
task = next(runner.define_tasks_from_config(exp.current_config))
task['sc_matrix'] = item[0][1]['sc_matrix']
params = runner.prepare_hdmf_params(task=task,seed=1)

2025-09-22 14:15:52,587 - hdmf_experiment_Fitting_Grid - INFO - Loading homeostatic fit results from /network/iss/home/ivan.mindlin/Repos/fastHDMF/data/fit_res_3-44.npy


2025-09-22 14:21:09,010 - hdmf_experiment_Fitting_Grid - INFO - Setting homeostatic DECAY=36158753912335801562119012352.00000 for LR=1.00000


## Get empirical data

In [7]:

from scipy.io import loadmat
emp_data = loadmat('../data/ts_coma24_AAL_symm_withSC.mat')
emp_data = np.squeeze(np.array(emp_data['timeseries_CNT24_symm']))
emp_data = np.stack(emp_data.flatten())[:,:,exp.current_config['simulation']['burnout']:]
isubfcd = np.triu_indices(params['N'], 1)
params["wsize"] = 30
params["overlap"] = 29

emp_fcds = []
for sub in range(emp_data.shape[0]):
    fcds = compute_fcd(emp_data[sub, :, :].T, params["wsize"], params['overlap'], isubfcd)
    upp_tr_fcd = np.corrcoef(fcds.T)
    emp_fcds.append(upp_tr_fcd)

emp_fc = np.mean(np.array([np.corrcoef(emp_data[sub, :, :]) for sub in range(emp_data.shape[0])]), axis=0)

# remove the diagonal

emp_fc = emp_fc - np.eye(emp_fc.shape[0])

emp_fcd = np.hstack((emp_fcds))

In [8]:
res

{'metadata': {'experiment_id': 'Fitting_Grid',
  'original_config_path': '/network/iss/home/ivan.mindlin/Repos/fastHDMF/configs/experiments/Fitting_Grid.yaml',
  'start_time': '2025-09-22T13:16:20.160444',
  'config': {'simulation': {'nb_steps': 500000,
    'obj_rate': 3.44,
    'with_decay': True,
    'with_plasticity': True,
    'TR': 2.4,
    'flp': 0.01,
    'fhp': 0.1,
    'burnout': 8,
    'over': {'seed': {'fun': 'np.arange', 'args': [1, 30, 1]}},
    'parallel': True},
   'grid': {'lrj': {'fun': 'np.logspace', 'args': [0, 5, 2]},
    'G': {'fun': 'np.arange', 'args': [0.1, 8.1, 2]}},
   'data': {'test_mode': False, 'max_subjects_test': 1, 'sc_root': 'AAL'},
   'output': {'save_full_outputs': True,
    'observables': [{'name': 'fc', 'signal': 'bold'},
     {'name': 'fcd', 'signal': 'bold'}]},
   'experiment': {'description': 'FC and FCD grid ', 'version': '1.0'}},
  'status': 'initialized',
  'observables_spec': [{'name': 'fc',
    'signal': ['bold'],
    'params': {'zero_diag':